In [40]:
%pip install transformers googletrans==4.0.0-rc1 nltk h5py json urllib

import json
import os
import numpy as np

import torch
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel

from PIL import Image
from googletrans import Translator
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import h5py

from tqdm import tqdm
import time

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
data_folder = os.path.join(os.getcwd(), "data")
os.makedirs(data_folder, exist_ok=True)
img_path = os.path.join(data_folder, "images.zip")
cap_path = os.path.join(data_folder, "captions.json")

'''if not os.path.exists(img_path):
    img_url = "https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/220ffc5a-8645-4c0b-bd1b-97188836ed3b/file_downloaded"
    img_path, _ = urllib.request.urlretrieve(img_url, img_path)
    print(f"Downloaded {img_path}")
else:
    print(f"File {img_path} already exists.")'''

if not os.path.exists(cap_path):
    cap_url = "https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/9b3e789a-5a5c-48b3-8a2c-2c91e9307c2a/file_downloaded"
    req = urllib.request.Request(cap_url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req) as response, open(cap_path, 'wb') as out_file:
        out_file.write(response.read())
    print(f"Downloaded {cap_path}")
else:
    print(f"File {cap_path} already exists.")

File c:\Users\shayo\OneDrive\Documents\ImageRetrieval-CNN-BERT\data\captions.json already exists.


In [42]:
if os.path.exists(cap_path):
    with open(cap_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("Preview of captions.json content:")
    print(json.dumps(data, indent=2)[:300])

Preview of captions.json content:
[
  {
    "filename": "1.png",
    "caption": [
      "\u09a4\u09bf\u09a8 \u099c\u09a8 \u09ae\u09c7\u09df\u09c7 \u09ae\u09be\u09a8\u09c1\u09b7 \u0986\u099b\u09c7\u0964 \u098f\u0995 \u099c\u09a8 \u09a6\u09be\u09dc\u09bf\u09df\u09c7 \u0986\u099b\u09c7 \u0986\u09b0 \u09a6\u09c1\u0987 \u099c\u09a8 \u09a


In [43]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shayo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shayo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
def batch_translate(captions, delimiter="|||"):
    """Translate captions in a batch using a delimiter."""
    translator = Translator()

    if not captions:
        raise ValueError("Caption batch is empty.")

    valid_captions = [caption for caption in captions if caption]
    
    if not valid_captions:
        raise ValueError("All captions in the batch are empty or None.")

    try:
        concatenated = f" {delimiter} ".join(captions)
        translated = translator.translate(concatenated, src='bn', dest='en')

        if not translated or not translated.text:
            raise ValueError("Translation returned no results.")
        time.sleep(2)
        return translated.text.split(delimiter)
    except Exception as e:
        print(f"Error during translation: {e}")
        return [""] * len(captions)

In [45]:
def preprocess_text(text):
    REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
    SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
    NUMBERS = re.compile(r'\d+')
    STOPWORDS = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = SYMBOLS_RE.sub('', text)
    text = NUMBERS.sub('', text)

    tokens = [word for word in text.split() if word not in STOPWORDS]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

In [46]:
if os.path.exists(cap_path):
    with open(cap_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")
    model.eval()

    batch_size = 100
    all_captions = [entry['caption'][0] for entry in data]
    filenames = [entry['filename'] for entry in data]

    with h5py.File("caption_embeddings.h5", "w") as hdf:
        for i in tqdm(range(0, len(all_captions), batch_size), desc="Processing batches"):
            batch_captions = all_captions[i:i + batch_size]
            batch_filenames = filenames[i:i + batch_size]

            translated_captions = batch_translate(batch_captions)

            for filename, english_caption in zip(batch_filenames, translated_captions):
                cleaned_tokens = preprocess_text(english_caption)
                tokenized_input = tokenizer(" ".join(cleaned_tokens), return_tensors="pt", padding=True, truncation=True)

                with torch.no_grad():
                    outputs = model(**tokenized_input)
                    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

                hdf.create_dataset(filename, data=embedding)

Processing batches:  21%|██        | 19/92 [02:15<08:28,  6.97s/it]

Error during translation: the JSON object must be str, bytes or bytearray, not NoneType


Processing batches:  22%|██▏       | 20/92 [02:19<07:16,  6.06s/it]

Error during translation: the JSON object must be str, bytes or bytearray, not NoneType


Processing batches:  23%|██▎       | 21/92 [02:23<06:25,  5.43s/it]

...

Processing batches:  97%|█████████▋| 89/92 [10:11<00:14,  4.99s/it]

Error during translation: the JSON object must be str, bytes or bytearray, not NoneType


Processing batches:  98%|█████████▊| 90/92 [10:15<00:09,  4.64s/it]

Error during translation: the JSON object must be str, bytes or bytearray, not NoneType


Processing batches: 100%|██████████| 92/92 [10:23<00:00,  6.78s/it]


In [47]:
with h5py.File("caption_embeddings.h5", "r") as hdf:
    for i in range(1, 11):
        filename = f"{i}.png"
        if filename in hdf:
            embed_np = hdf[filename][:]
            embed = torch.tensor(embed_np)
            print(f"Embedding for {filename}, shape {embed.shape}:\n", embed)
        else:
            print(f"Embedding for {filename} not found.")

Embedding for 1.png, shape torch.Size([768]):
 tensor([-1.8174e-01,  1.5768e-01, -7.6489e-02, -6.2967e-02, -5.9036e-02,
        -3.1719e-01,  5.7200e-02,  3.0386e-01, -2.0034e-01, -5.9699e-01,
         1.4649e-01, -2.5877e-01,  5.6815e-02,  5.1283e-01, -4.7342e-01,
        -2.3066e-01,  5.0184e-02,  2.3710e-01, -2.2853e-01,  1.2698e-01,
         2.6595e-01, -3.5752e-01,  3.8996e-02, -4.8054e-01,  3.9018e-03,
        -2.4877e-01,  2.5391e-01,  7.5803e-02,  5.1177e-01,  1.3827e-01,
        -1.2290e-01,  5.2137e-02, -3.1580e-01,  1.0608e-02, -2.1062e-01,
        -4.7340e-01, -5.5973e-02,  5.6410e-01, -1.8174e-01, -7.7918e-02,
        -5.1626e-01, -6.1322e-01, -4.7361e-03, -1.5094e-01, -2.2534e-01,
        -3.3963e-01, -1.5403e-01,  1.8549e-01,  1.4138e-01,  2.3144e-01,
        -4.2835e-01, -1.2070e-02, -8.9260e-02,  1.2281e-01,  6.6684e-02,
         2.3808e-01, -3.4825e-03, -2.0153e-02,  1.1176e-02, -5.7880e-02,
         4.4596e-01,  1.0899e-01,  3.2055e-02, -3.7851e-01,  1.5599e-01,
    